In [6]:
import random
import numpy as np
from string import ascii_lowercase
from itertools import combinations

In [9]:
def new_problem(n, k, m):
    positive_var = list(ascii_lowercase)[:n]
    negative_var = [var.upper() for var in positive_var]
    variables = positive_var + negative_var
    problem = []
    threshold = 10       
    i = 0
    comb = list(combinations(variables, k))
    while i < threshold:
        c = random.sample(comb, m)
        if c not in problem:
            problem.append(c)
            i += 1
    
    problems_new = []
    for c in problem:
        temp = [list(sub) for sub in c]
        problems_new.append(temp)
    return  variables,problems_new 

In [10]:
def assign_randome_values(variables, n):
    litral = list(np.random.choice(2, n))
    negation = [abs(i - 1) for i in litral]
    assign = litral + negation
    return dict(zip(variables,assign))

In [11]:
def heruistic(problem, assign):
    count = 0
    for sub in problem:       
        encode = [assign[val] for val in sub]
        count += any(encode)
    return count

In [12]:
def next_node(current):
    key = list(current.keys())
    key.sort()
    key = key[0 : len(key) // 2]
    successors = []
    for k in key:
        temp = current.copy()
        temp[k] = abs(temp[k] - 1)
        temp[chr(ord(k) + 32)] = abs(temp[chr(ord(k) + 32)] - 1)
        successors.append(temp)  
    return successors

In [13]:
print(next_node({'a': 0, 'b': 0, 'c': 0, 'A': 1, 'B': 1, 'C': 1}))

[{'a': 1, 'b': 0, 'c': 0, 'A': 0, 'B': 1, 'C': 1}, {'a': 0, 'b': 1, 'c': 0, 'A': 1, 'B': 0, 'C': 1}, {'a': 0, 'b': 0, 'c': 1, 'A': 1, 'B': 1, 'C': 0}]


In [14]:
def select_node(success, problem):
    heruistic_val = []
    for i in success:
        heruistic_val.append(heruistic(problem, i))
    index = heruistic_val.index(max(heruistic_val))
    return success[index]

In [15]:
def check_goal_state(state, problem):
    count = 0
    for sub in problem:       
        encode = [state[val] for val in sub]
        count += any(encode)
    return len(problem) == count

In [23]:
def hill_climbing(current_state, problem, step):
    if step == 10000:
        print('Search halted at depth:', step)
        return
    if check_goal_state(current_state,problem):
        print(current_state,step)
        return 
    else:
        step += 1
        successor = next_node(current_state)
        new_node = select_node(successor, problem)
        hill_climbing(new_node,problem, step)

In [24]:
n = 2
k = 3
m = 2
var, prob = new_problem(n, k, m)
print(var)
for i in prob:
    print(i)


['a', 'b', 'A', 'B']
[['a', 'A', 'B'], ['a', 'b', 'B']]
[['a', 'A', 'B'], ['a', 'b', 'A']]
[['a', 'b', 'B'], ['a', 'A', 'B']]
[['b', 'A', 'B'], ['a', 'b', 'B']]
[['b', 'A', 'B'], ['a', 'A', 'B']]
[['b', 'A', 'B'], ['a', 'b', 'A']]
[['a', 'b', 'B'], ['b', 'A', 'B']]
[['a', 'b', 'A'], ['a', 'A', 'B']]
[['a', 'b', 'A'], ['a', 'b', 'B']]
[['a', 'b', 'A'], ['b', 'A', 'B']]


In [16]:
start_state = assign_randome_values(var, n)
print(start_state,prob[0])
hill_climbing(start_state, prob[0], 0)

{'a': 1, 'b': 0, 'A': 0, 'B': 1} [['b', 'A', 'B'], ['a', 'A', 'B']]
{'a': 1, 'b': 0, 'A': 0, 'B': 1} 0


In [17]:
print(check_goal_state({'a': 0, 'b': 1, 'c': 0, 'd': 1, 'A': 1, 'B': 0, 'C': 1, 'D': 0}, prob[7]))

True


# Beam search 

In [18]:
def heruistic(problem, assign):
    count = 0
    for sub in problem:       
        encode = [assign[val] for val in sub]
        count += any(encode)
    return count

In [19]:
def select_node_beam(success, problem, beamwidth):
    heruistic_val = []
    beam_nodes = []
    for i in success:
        heruistic_val.append(heruistic(problem, i))
    for i in range(beamwidth):
        index = heruistic_val.index(max(heruistic_val))
        beam_nodes.append(success[index])
        success.remove(success[index])
        
    return beam_nodes

In [20]:
def check_goal_state(state, problem, beamwidth):
    count = 0
    for sub in problem:       
        encode = [state[val] for val in sub]
        count += any(encode)
    return len(problem) == count

In [21]:
def next_node_beam(success):
    nextnode = []
    for current in success:
        key = list(current.keys())
        key.sort()
        key = key[0 : len(key) // 2]
        successors = []
        for k in key:
            temp = current.copy()
            temp[k] = abs(temp[k] - 1)
            temp[chr(ord(k) + 32)] = abs(temp[chr(ord(k) + 32)] - 1)
            successors.append(temp)
            nextnode.append(temp)
    return nextnode

In [39]:
def beam_search(start, problem, step, beamwidth):
    if step == 10000:
        print('Search halted at depth:', step)
        return
    for current_state in start:
        print(current_state)
        if check_goal_state(current_state, problem, beamwidth):
            print(current_state, step)
            return 
    step += 1
    successor = next_node_beam(start, problem, beamwidth)
    select_node_beam(successor, prob[0], beamwidth)
    beam_search(start, problem, step, beamwidth)

In [40]:
n = 5
var, probl = new_problem(n, 3, 5)
print(var)
for i in probl:
    print(i)

['a', 'b', 'c', 'd', 'e', 'A', 'B', 'C', 'D', 'E']
[['b', 'e', 'B'], ['a', 'e', 'C'], ['a', 'b', 'C'], ['c', 'd', 'e'], ['b', 'd', 'A']]
[['a', 'A', 'B'], ['b', 'D', 'E'], ['a', 'b', 'E'], ['a', 'd', 'C'], ['C', 'D', 'E']]
[['a', 'A', 'C'], ['c', 'A', 'D'], ['a', 'b', 'e'], ['a', 'b', 'c'], ['c', 'C', 'E']]
[['a', 'b', 'D'], ['a', 'd', 'C'], ['b', 'C', 'E'], ['e', 'A', 'E'], ['a', 'c', 'B']]
[['d', 'e', 'C'], ['a', 'c', 'E'], ['e', 'C', 'E'], ['a', 'c', 'd'], ['b', 'e', 'E']]
[['b', 'd', 'A'], ['a', 'A', 'E'], ['d', 'A', 'D'], ['b', 'c', 'e'], ['a', 'd', 'C']]
[['a', 'c', 'A'], ['b', 'c', 'C'], ['b', 'A', 'B'], ['a', 'D', 'E'], ['b', 'd', 'D']]
[['d', 'D', 'E'], ['a', 'b', 'C'], ['A', 'C', 'D'], ['e', 'A', 'D'], ['b', 'A', 'B']]
[['c', 'e', 'E'], ['B', 'D', 'E'], ['d', 'A', 'D'], ['a', 'A', 'B'], ['a', 'b', 'E']]
[['a', 'd', 'E'], ['e', 'A', 'D'], ['a', 'b', 'e'], ['a', 'd', 'B'], ['e', 'A', 'E']]


In [41]:
start_state = assign_randome_values(var, n)
print([start_state, start_state])

[{'a': 0, 'b': 1, 'c': 0, 'd': 0, 'e': 0, 'A': 1, 'B': 0, 'C': 1, 'D': 1, 'E': 1}, {'a': 0, 'b': 1, 'c': 0, 'd': 0, 'e': 0, 'A': 1, 'B': 0, 'C': 1, 'D': 1, 'E': 1}]


In [42]:
beam_search([start_state, start_state], prob[2], 0, 1)

{'a': 0, 'b': 1, 'c': 0, 'd': 0, 'e': 0, 'A': 1, 'B': 0, 'C': 1, 'D': 1, 'E': 1}
{'a': 0, 'b': 1, 'c': 0, 'd': 0, 'e': 0, 'A': 1, 'B': 0, 'C': 1, 'D': 1, 'E': 1} 0


# Variable Neighborhood Descent

In [43]:
def heruistic(problem, assign):
    count = 0
    for sub in problem:       
        encode = [assign[val] for val in sub]
        count += any(encode)
    return count

In [44]:
def check_goal_state(state, problem):
    count = 0
    for sub in problem:       
        encode = [state[val] for val in sub]
        count += any(encode)
    return len(problem) == count

In [45]:
def select_node(success, problem):
    heruistic_val = []
    for i in success:
        heruistic_val.append(heruistic(problem, i))
    index = heruistic_val.index(max(heruistic_val))
    return success[index]

In [46]:
def nghd1(current):
    key = list(current.keys())
    key.sort()
    key = key[0 : len(key) // 2]
    successors = []
    for k in key:
        temp = current.copy()
        temp[k] = abs(temp[k] - 1)
        temp[chr(ord(k) + 32)] = abs(temp[chr(ord(k) + 32)] - 1)
        successors.append(temp)   
    return successors

In [47]:
print(nghd1({'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 1, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}))

[{'a': 0, 'b': 1, 'c': 1, 'd': 1, 'e': 1, 'A': 1, 'B': 0, 'C': 0, 'D': 0, 'E': 0}, {'a': 1, 'b': 0, 'c': 1, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 0, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 1}]


In [48]:
def nghd2(current):
    key = list(current.keys())
    key.sort()
    key = key[0 : len(key) // 2]
    successors = []
    for j in range(0,len(key) - 1):
        for i in range(j,len(key) - 1):
            temp = current.copy()
            temp[key[i]] = abs(temp[key[i]] - 1)
            temp[key[i + 1]] = abs(temp[key[i + 1]] - 1)
            temp[chr(ord(key[i]) + 32)] = abs(temp[chr(ord(key[i])+32)]-1)
            temp[chr(ord(key[i + 1]) + 32)] = abs(temp[chr(ord(key[i + 1]) + 32)] - 1)
            successors.append(temp)   
    return successors

In [49]:
print(nghd2({'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 1, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}))

[{'a': 0, 'b': 0, 'c': 1, 'd': 1, 'e': 1, 'A': 1, 'B': 1, 'C': 0, 'D': 0, 'E': 0}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}, {'a': 1, 'b': 1, 'c': 1, 'd': 0, 'e': 0, 'A': 0, 'B': 0, 'C': 0, 'D': 1, 'E': 1}]


In [50]:
def nghd3(current):
    key = list(current.keys())
    key.sort()
    key = key[0 : len(key) // 2]
    successors = []
    for j in range(0,len(key)-2):
        for k in range(j,len(key)-2):
            for i in range(k,len(key)-2):               
                temp = current.copy()
                temp[key[i]] = abs(temp[key[i]] - 1)
                temp[key[i+1]] = abs(temp[key[i + 1]] - 1)
                temp[chr(ord(key[i]) + 32)] = abs(temp[chr(ord(key[i]) + 32)] - 1)
                temp[chr(ord(key[i + 1]) + 32)] = abs(temp[chr(ord(key[i + 1]) + 32)] - 1)
                successors.append(temp)   
    return successors

In [51]:
print(nghd3({'a': 1, 'b': 1, 'c': 1, 'd': 1, 'e': 1, 'A': 0, 'B': 0, 'C': 0, 'D': 0, 'E': 0}))

[{'a': 0, 'b': 0, 'c': 1, 'd': 1, 'e': 1, 'A': 1, 'B': 1, 'C': 0, 'D': 0, 'E': 0}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 0, 'c': 0, 'd': 1, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 0, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 1, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 0, 'C': 1, 'D': 1, 'E': 0}]


In [53]:
def Variable_Neighborhood1(current_state,problem,step):
    if step == 10000:
        print('Search halted at depth:',step)
        return
    if check_goal_state(current_state,problem):
        return current_state
    else:
        step += 1
        successor = next_node(current_state)
        new_node = nghd1(successor, problem)
        Variable_Neighborhood1(new_node, problem, step)

In [55]:
def Variable_Neighborhood2(current_state,problem,step):
    if step == 10000:
        print('Search halted at depth:', step)
        return
    if check_goal_state(current_state,problem):
        return current_state
    else:
        step += 1
        successor = next_node(current_state)
        new_node = nghd2(successor, problem)
        Variable_Neighborhood2(new_node, problem, step)

In [56]:
def Variable_Neighborhood3(current_state,problem,step):
    if step == 10000:
        print('Search halted at depth:', step)
        return
    if check_goal_state(current_state,problem):
        print(current_state,step)
        return 
    else:
        step += 1
        successor = next_node(current_state)
        new_node = nghd3(successor, problem)
        Variable_Neighborhood2(new_node, problem, step)

In [58]:
n = 5
var, probl = new_problem(5, 3, 5)
print(var)
for i in probl:
    print(i)

['a', 'b', 'c', 'd', 'e', 'A', 'B', 'C', 'D', 'E']
[['a', 'b', 'C'], ['e', 'B', 'C'], ['a', 'c', 'D'], ['d', 'e', 'E'], ['B', 'D', 'E']]
[['e', 'C', 'D'], ['b', 'D', 'E'], ['d', 'e', 'C'], ['e', 'A', 'B'], ['a', 'c', 'B']]
[['B', 'C', 'E'], ['b', 'e', 'E'], ['b', 'A', 'E'], ['b', 'C', 'E'], ['b', 'c', 'B']]
[['b', 'e', 'A'], ['d', 'D', 'E'], ['b', 'c', 'e'], ['d', 'A', 'B'], ['b', 'c', 'd']]
[['c', 'e', 'E'], ['b', 'B', 'C'], ['a', 'b', 'c'], ['e', 'B', 'D'], ['c', 'e', 'A']]
[['e', 'D', 'E'], ['c', 'B', 'C'], ['b', 'e', 'B'], ['d', 'A', 'E'], ['a', 'B', 'C']]
[['a', 'e', 'D'], ['d', 'A', 'D'], ['d', 'e', 'E'], ['a', 'c', 'e'], ['a', 'b', 'c']]
[['B', 'C', 'D'], ['a', 'd', 'C'], ['a', 'd', 'e'], ['e', 'B', 'C'], ['c', 'A', 'D']]
[['b', 'd', 'B'], ['b', 'e', 'E'], ['a', 'c', 'D'], ['c', 'd', 'D'], ['a', 'C', 'D']]
[['B', 'D', 'E'], ['d', 'D', 'E'], ['b', 'c', 'e'], ['b', 'A', 'B'], ['a', 'C', 'E']]


In [62]:
start_state = assign_randome_values(var, n)
print([start_state, start_state])

[{'a': 1, 'b': 0, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 1, 'E': 0}, {'a': 1, 'b': 0, 'c': 0, 'd': 0, 'e': 1, 'A': 0, 'B': 1, 'C': 1, 'D': 1, 'E': 0}]


In [63]:
var1 = start_state
var2 = Variable_Neighborhood1(var1, prob[0], 0)
var3 = Variable_Neighborhood2(var2, prob[0], 0)
Variable_Neighborhood2(var3, prob[0], 0)

{'a': 1,
 'b': 0,
 'c': 0,
 'd': 0,
 'e': 1,
 'A': 0,
 'B': 1,
 'C': 1,
 'D': 1,
 'E': 0}